In [2]:
import os
from smolagents import AzureOpenAIServerModel, TransformersModel

MODE = 'azure'  # or 'azure'
if MODE == 'azure':
    model = AzureOpenAIServerModel(
        model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
        api_version=os.environ.get("OPENAI_API_VERSION", "")
    )
if MODE == 'local':
    model = TransformersModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    )

In [1]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "pre_medications").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [3]:
from medminer.task.medication import medication_task

for doc in docs[:1]:
    medication_task.run(model, doc.content)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Task name: medication                                                                                           │
│ Prompt:                                                                                                         │
│     Given a list of medications, save all medications for the patient as csv.                                   │
│     To complete the task make the following steps:                                                              │
│     1. extract all information defined in the columns below from the document. Infer the                        │
│ medication_name_corrected column.                                                                               │
│     2. get the RXCUI for all medications. Use the corrected name of the medications.                            │
│     3. get the ATC code for all medications. Use the rxcui of the medications.                                  │
│     4. save the medication information as csv with the columns defined below.                                   │
│                                                                                                                 │
│     Columns:                                                                                                    │
│     - patient_id: The patient ID.                                                                               │
│     - medication_name: The name of the medication in the document without dose, unit or additional information. │
│     - medication_name_corrected: Use the following format "Brand name or medication name (active ingredient)".  │
│ e.g. "Aspirin (acetylsalicylic acid)".                                                                          │
│     - dose: The dose of the medication. this sould only contain the numeric value.                              │
│     - unit: The unit of the dose (e.g. ml, mg, ...). if not applicable, write an empty string.                  │
│     - dosage_morning: The dose in the morning. if not applicable, write a 0.                                    │
│     - dosage_noon: The dose in the noon. if not applicable, write a 0.                                          │
│     - dosage_evening: The dose in the evening. if not applicable, write a 0.                                    │
│     - dosage_night: The dose in the night. if not applicable, write a 0.                                        │
│     - dosage_information: Additional information about the dosage. if not applicable, write an empty string.    │
│     - rxcui: The RXCUI of the medication. if not applicable, write an empty string.                             │
│     - atc_id: The ATC code of the medication. if not applicable, write an empty string.                         │
│     - atc_name: The name of the ATC code. if not applicable, write an empty string.                             │
│     - atc_type: The type of the ATC code. if not applicable, write an empty string.                             │
│                                                                                                                 │
│                                                                                                                 │
│ --------------------------------------------------------------------------------                                │
│ Data:                                                                                                           │
│     Patiend: 5                                                                                                  │
│     "Amlodipin 10mg   1-0-0                                                                                     │
│     \                                                 

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating or parsing output:
Connection error.

[Step 1: Duration 1.45 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 20 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating or parsing output:
Connection error.

[Step 20: Duration 1.32 seconds]

Reached max steps.

[Step 21: Duration 2.76 seconds]